In [6]:
pip install biopython

In [7]:
from Bio import Align
import numpy as np

#### 1. Pairwise Alignment using Smith-Waterman (Local Alignment)

In [8]:
# Initialize the aligner with match and mismatch scores
aligner = Align.PairwiseAligner()
aligner.match_score = 1.0
aligner.mismatch_score = -2.0
aligner.open_gap_score = -1
aligner.extend_gap_score = 0
aligner.mode = "local"  # Set for local alignment (Smith-Waterman)

In [9]:
# Perform local alignment
target = "GAACT"
query = "GAT"
alignments = aligner.align(target, query)
print(f"Score (Local Alignment): {aligner.score(target, query)}")
for alignment in alignments:
    print(alignment)

Score (Local Alignment): 2.0
target            0 GA 2
                  0 || 2
query             0 GA 2



#### 2. Pairwise Alignment using Needleman-Wunsch (Global Alignment)

In [10]:
# Initialize the aligner for global alignment
aligner.mode = "global"  # Set for global alignment (Needleman-Wunsch)
aligner.mismatch_score = -10

In [11]:
# Perform global alignment
alignments_global = aligner.align("AAACAAA", "AAAGAAA")
print(f"\nGlobal Alignment Score: {aligner.score('AAACAAA', 'AAAGAAA')}")
print(alignments_global[0])  # Print first alignment


Global Alignment Score: 4.0
target            0 AAA-CAAA 7
                  0 |||--||| 8
query             0 AAAG-AAA 7



#### 3. Needleman-Wunsch Algorithm for Global Alignment

In [12]:
def nw(x, y, match=1, mismatch=1, gap=1):
    """Custom Needleman-Wunsch algorithm for global sequence alignment."""
    nx, ny = len(x), len(y)
    F = np.zeros((nx + 1, ny + 1))  # Scoring matrix
    F[:,0] = np.arange(0, -nx * gap - 1, -gap)
    F[0,:] = np.arange(0, -ny * gap - 1, -gap)

    # Fill the scoring matrix
    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            match_score = F[i - 1, j - 1] + (match if x[i - 1] == y[j - 1] else -mismatch)
            delete = F[i - 1, j] - gap
            insert = F[i, j - 1] - gap
            F[i, j] = max(match_score, delete, insert)

    # Traceback
    i, j = nx, ny
    aligned_x, aligned_y = [], []
    while i > 0 or j > 0:
        current_score = F[i, j]
        if i > 0 and j > 0 and current_score == F[i - 1, j - 1] + (match if x[i - 1] == y[j - 1] else -mismatch):
            aligned_x.append(x[i - 1])
            aligned_y.append(y[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and current_score == F[i - 1, j] - gap:
            aligned_x.append(x[i - 1])
            aligned_y.append('-')
            i -= 1
        else:
            aligned_x.append('-')
            aligned_y.append(y[j - 1])
            j -= 1

    # Reverse alignments to get correct order
    return ''.join(reversed(aligned_x)), ''.join(reversed(aligned_y))

In [13]:
# Example sequences for global alignment using custom Needleman-Wunsch
seq1 = "GATTACA"
seq2 = "GCATGCU"
alignment1, alignment2 = nw(seq1, seq2)
print(f"\nNeedleman-Wunsch Alignment:\n{alignment1}\n{alignment2}")


Needleman-Wunsch Alignment:
G-ATTACA
GCA-TGCU
